In [1]:
#function to write in csv
import csv
def write_list_in_file(final, name):
    with open(name, "w", newline="",encoding="utf8") as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(final)


In [2]:
#Function to read csv files
from csv import reader
# Load a CSV file\n",
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


In [6]:
import numpy as np
base_data = load_csv('season1.csv')    
base_data = np.array(base_data[1:])

In [15]:
#learn number of clusters using silhouette_score

from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

num_clusters = 0
max_silhouette = -1

for n_clusters in range(2,10):

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(base_data)

    silhouette_avg = silhouette_score(base_data, cluster_labels)
    #print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)
    
    if(silhouette_avg > max_silhouette):
        max_silhouette = silhouette_avg
        num_clusters = n_clusters

print("Number of Clusters: ",num_clusters)


Number of Clusters:  2


In [19]:
#learn centers from source/base model
import numpy as np
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(base_data)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
#print(kmeans.cluster_centers_)
print(kmeans.inertia_)

5.87452641436


In [20]:
#no of elements and load target data
elements = [10,20,30,40,50]
target_data = load_csv('season2.csv')

In [25]:
#base model results
kmeans = KMeans(n_clusters=num_clusters, init=base_centroids, max_iter=1)
kmeans.fit(target_data[1:])
print(kmeans.inertia_)

6.88333196231


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [26]:
#retraining results
#retrained_centroids= []

for n in elements:

    #Read Available Limited Target Data
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(target_data[1:n+1])
    
    #Result on whole Target Data
    kmeans = KMeans(n_clusters=num_clusters, init=kmeans.cluster_centers_, max_iter=1)
    kmeans.fit(target_data[1:])

    print(kmeans.inertia_)

6.89377210019
6.72197078224
6.65951589037
6.5807709417
6.58887790128


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
